In [1]:
import numpy as np 
import re
import datetime as dt 
import matplotlib.pyplot as plt 
import pandas as pd
import sys
from tqdm import tqdm   
sys.path.append('/Users/jerald/Documents/Dir/Python/stocker')
from models.sim.lsm import OptionSim as sim

def prep_df(df, rf_rate = 0.45, nsims = 100):
    df['rf']  = rf_rate
    df['nsim'] = nsims
    df['impliedvolatility'] = df.impliedvolatility.astype(float)
    df['strike'] = df.strike.astype(float)
    df['stk_price'] = df.stk_price.astype(float)
    df['lastprice'] = df.lastprice.astype(float)
    df.expiry = pd.to_datetime(df.expiry)
    df.gatherdate = pd.to_datetime(df.gatherdate)
    df['dte'] = ((df.expiry - df.gatherdate).dt.days).astype(float) 
    # df['timevalue'] = df.timevalue.astype(float) / 1e7
    df['timevalue'] = df.dte / 252
    return df


    
def lsm(df, rf_rate = 0.0379, nsims = 1000, jump = False):
    df = prep_df(df, rf_rate, nsims) 
    lodf = []
    vals = list(zip(df.stk_price, df.strike, df.rf, df.dte,df.timevalue, df.impliedvolatility,df.type, df.nsim ,df.lastprice))
    for s, k, r, dte, tv,  sig, ty, nsim, lp in tqdm(vals):
        s = sim(s, k, r, dte, tv, sig, ty, nsim, lp)
        lodf.append(s.run(jump = jump))
    
    return pd.concat(lodf)



from bin.main import Manager 
m = Manager('../../')

Options db Connected: 2024-11-13 12:52:45.441765
Prices Connected: 2024-11-13 12:52:45.442638


In [12]:
df =pd.read_sql('select * from spy where datetime(gatherdate) = (select max(datetime(gatherdate)) from spy) ', m.Optionsdb.change_db)
df

,gatherdate,contractsymbol,stk_price,lastprice,ask,bid,change,percentchange,volume,openinterest,...,lastprice_avg_5d,pct_chg,iv_chg,iv_avg_5d,iv_avg_30d,iv_avg_all,vol_chg,oi_chg,flag,amnt
0,2024-11-13 11:48:42,SPY241113C00300000,598.02,297.28,297.78,297.04,-2.209992,-0.737918,1,4,...,298.385,-0.737921,-3.359367,1.679693,1.679693,1.679693,-1.0,0.0,0,0
1,2024-11-13 11:48:42,SPY241113C00455000,598.02,142.28,142.88,142.08,0.039993,0.028117,2,2,...,142.260,0.028121,-1.488274,0.744147,0.744147,0.744147,0.0,0.0,0,0
2,2024-11-13 11:48:42,SPY241113C00500000,598.02,98.99,97.78,96.92,0.000000,0.000000,5,15,...,98.990,0.000000,-1.013667,0.506843,0.506843,0.506843,0.0,5.0,0,0
3,2024-11-13 11:48:42,SPY241113C00520000,598.02,64.12,77.86,77.16,0.000000,0.000000,0,0,...,64.120,0.000000,-0.842765,0.421393,0.421393,0.421393,0.0,0.0,0,0
4,2024-11-13 11:48:42,SPY241113C00530000,598.02,67.47,67.72,67.07,-0.639999,-0.939656,220,230,...,67.015,1.367187,-0.734368,0.367194,0.367194,0.367194,10.0,230.0,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7846,2024-11-13 11:48:42,SPY270115P00685000,598.02,105.25,90.50,86.95,0.000000,0.000000,1,0,...,105.250,0.000000,0.003143,0.072839,0.072839,0.072839,0.0,0.0,0,0
7847,2024-11-13 11:48:42,SPY270115P00690000,598.02,92.31,95.00,91.13,0.000000,0.000000,12,12,...,92.310,0.000000,0.003448,0.072473,0.072473,0.072473,0.0,0.0,0,0
7848,2024-11-13 11:48:42,SPY270115P00700000,598.02,100.78,105.14,100.15,0.000000,0.000000,6,0,...,100.780,0.000000,0.005646,0.077600,0.077600,0.077600,0.0,0.0,0,0
7849,2024-11-13 11:48:42,SPY270115P00870000,598.02,292.17,274.29,269.50,0.000000,0.000000,0,0,...,292.170,0.000000,0.000397,0.144982,0.144982,0.144982,0.0,0.0,0,0


In [13]:
#  Simulate for an Entire Stock 
df =pd.read_sql('select * from spy where datetime(gatherdate) = (select max(datetime(gatherdate)) from spy) ', m.Optionsdb.change_db)


df = m.Optionsdb.pcdb(df) 
df['moneyness'] = df.stk_price / df.strike
df['dte'] = (pd.to_datetime(df.expiry) - pd.to_datetime(df.gatherdate)).dt.days

df = df[(df.moneyness.between(0.65, 1.35)) & (df.dte.between(10, 40)) & (df.lastprice > 0.05)].dropna()
ind = df[['contractsymbol','gatherdate']]

# Simulate For An Entire Stock 
est = lsm(df, rf_rate = 0.0379, nsims = 1000, jump = True)
est['contractsymbol'] = df.contractsymbol.values
est['avg_est'] = est[['LSMC Normal', 'LSMC Poly', 'LSMC CV', 'MC']].mean(axis = 1).round(2)
est = est.set_index('contractsymbol').reset_index()

100%|██████████| 979/979 [00:46<00:00, 21.22it/s]


In [14]:
df.contractsymbol.unique().shape

(979,)

In [16]:
est_df = m.Optionsdb.pcdb(est)[(est.Observed < est.avg_est) & (est.Observed.between(.25, .50))]
# Discount (or premium) percentage: The % difference between the option's market price and the model's price.
est_df['discount'] = ((est_df.Observed - est_df.avg_est) / est_df.Observed) * 100
est_df.sort_values('discount', ascending = False).head(20)

,contractsymbol,stock,type,strike,expiry,LSMC Normal,LSMC Poly,LSMC CV,LSM Primal,MC,BS,Observed,avg_est,discount
470,SPY241206P00533000,SPY,Put,533.0,2024-12-06,0.270106,0.446138,0.496199,0.466287,0.759776,0.677926,0.48,0.49,-2.083333
474,SPY241206P00537000,SPY,Put,537.0,2024-12-06,0.408130,0.552157,0.447536,0.712273,0.685748,0.717465,0.50,0.52,-4.000000
264,SPY241129P00561000,SPY,Put,561.0,2024-11-29,0.378652,0.622418,0.587499,0.716215,0.379922,0.634490,0.44,0.49,-11.363636
464,SPY241206P00527000,SPY,Put,527.0,2024-12-06,0.212445,0.416233,0.840088,0.476984,0.479805,0.625026,0.43,0.49,-13.953488
475,SPY241206P00538000,SPY,Put,538.0,2024-12-06,0.341784,0.579605,0.585060,0.764107,0.684284,0.713488,0.48,0.55,-14.583333
256,SPY241129P00553000,SPY,Put,553.0,2024-11-29,0.252308,0.392592,0.400182,0.426348,0.515109,0.513126,0.33,0.39,-18.181818
261,SPY241129P00558000,SPY,Put,558.0,2024-11-29,0.326591,0.554749,0.471371,0.484668,0.455427,0.573269,0.38,0.45,-18.421053
479,SPY241206P00542000,SPY,Put,542.0,2024-12-06,0.445002,0.597790,0.548421,0.660391,0.581382,0.764462,0.45,0.54,-20.000000
468,SPY241206P00531000,SPY,Put,531.0,2024-12-06,0.316190,0.524421,0.773053,0.569907,0.738095,0.649893,0.49,0.59,-20.408163
249,SPY241129P00546000,SPY,Put,546.0,2024-11-29,0.221334,0.394574,0.349174,0.383755,0.339175,0.436108,0.27,0.33,-22.222222


In [17]:
est_df[est_df.discount < 0].shape[0], est_df[est_df.discount > 0].shape[0], est_df.shape[0]

(79, 0, 79)

In [18]:
df[df['amnt'] > 0 ]

,contractsymbol,stock,type,strike,expiry,gatherdate,stk_price,lastprice,ask,bid,...,iv_avg_all,vol_chg,oi_chg,flag,amnt,moneyness,dte,rf,nsim,timevalue
1591,SPY241129C00500000,SPY,Call,500.0,2024-11-29,2024-11-13 11:48:42,598.02,98.90,99.10,98.64,...,0.503423,6.0,4.0,1,3,1.196040,15.0,0.0379,1000,0.059524
1744,SPY241129P00461000,SPY,Put,461.0,2024-11-29,2024-11-13 11:48:42,598.02,0.06,0.06,0.05,...,0.448736,5.0,6.0,1,4,1.297223,15.0,0.0379,1000,0.059524
1774,SPY241129P00491000,SPY,Put,491.0,2024-11-29,2024-11-13 11:48:42,598.02,0.11,0.10,0.09,...,0.368659,35.0,102.0,1,34,1.217963,15.0,0.0379,1000,0.059524
1780,SPY241129P00497000,SPY,Put,497.0,2024-11-29,2024-11-13 11:48:42,598.02,0.12,0.10,0.09,...,0.350348,119.0,292.0,1,118,1.203260,15.0,0.0379,1000,0.059524
1781,SPY241129P00498000,SPY,Put,498.0,2024-11-29,2024-11-13 11:48:42,598.02,0.10,0.11,0.10,...,0.349128,1.0,11.0,1,1,1.200843,15.0,0.0379,1000,0.059524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2621,SPY241220P00515000,SPY,Put,515.0,2024-12-20,2024-11-13 11:48:42,598.02,0.53,0.53,0.52,...,0.249519,-155.0,1488.0,1,1305,1.161204,36.0,0.0379,1000,0.142857
2631,SPY241220P00533000,SPY,Put,533.0,2024-12-20,2024-11-13 11:48:42,598.02,0.83,0.79,0.78,...,0.216622,19.0,34.0,1,4,1.121989,36.0,0.0379,1000,0.142857
2635,SPY241220P00537000,SPY,Put,537.0,2024-12-20,2024-11-13 11:48:42,598.02,0.89,0.88,0.87,...,0.209847,30.0,93.0,1,71,1.113631,36.0,0.0379,1000,0.142857
2669,SPY241220P00571000,SPY,Put,571.0,2024-12-20,2024-11-13 11:48:42,598.02,2.64,2.63,2.61,...,0.152123,47.0,180.0,1,70,1.047320,36.0,0.0379,1000,0.142857
